Download and move the PID data


In [ ]:
$ hdfs dfs -mkdir -p hive_02/pid_gtfs\
$ hdfs dfs -put PID_GTFS/* hive_02/pid_gtfs
$ hdfs dfs -ls hive_02/pid_gtfs

Found 16 items

-rw-r--r--   2 debian users        162 2022-10-25 09:00 hive_02/pid_gtfs/agency.txt

-rw-r--r--   2 debian users      18706 2022-10-25 09:00 hive_02/pid_gtfs/calendar.txt

-rw-r--r--   2 debian users     124035 2022-10-25 09:00 hive_02/pid_gtfs/calendar_dates.txt

## 1.


In [ ]:
$ hdfs dfs -mkdir -p ext_tables/routes
$ hdfs dfs -cp hive_02/pid_gtfs/routes.txt ext_tables/routes

In [ ]:
create external table route_ext,
( 
route_id string, 
agency_id string, 
route_short_name string,
route_long_name string,
route_type string,
route_url string,
route_color string,
route_text_color string,
is_night boolean,
is_regional boolean,
is_substitute_transport boolean,
)
row format delimited
fields terminated by ',',
lines terminated by '\n',
stored as textfile,
location '/user/debian/ext_tables/routes/',
tblproperties ("skip.header.line.count" = "1");


In [ ]:
select * from route_ext limit 10;

select count(*) from route_ext;

select count(*) from route_ext where route_short_name is NULL;


## 2.

In [ ]:
create table route
(
route_id string,
agency_id string,
route_short_name string,
route_long_name string,
route_type string,
route_url string,
route_color string,
route_text_color string,
is_night boolean,
is_regional boolean,
is_substitute_transport boolean
)
stored as parquet
tblproperties("parquet.compress"="SNAPPY");

In [ ]:
insert overwrite table route,
select,
route_id, agency_id, route_short_name, route_long_name, route_type,
route_url, route_color, route_text_color, is_night, is_regional, is_substitute_transport,
from route_ext;

select * from route limit 10;

select count(*) from route;

In [ ]:
hdfs dfs -du -s -h /user/hive/warehouse/debian.db  --result 74.4 K

describe formatted route; --shows in bytes (76139)

-- exteranl table size is 82.9K